In [10]:
from datasets import Dataset, DatasetDict
import numpy as np
import pandas as pd
import torch
import transformers
from sklearn.preprocessing import MultiLabelBinarizer
#https://towardsdatascience.com/adding-custom-layers-on-top-of-a-hugging-face-model-f1ccdfc257bd

In [2]:
###
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device()

(True, 1, 0)

In [3]:
df = pd.read_json('PyCodes.json')
df.head()
#data=load_dataset("json",data_files='PyCodes.json')

,Tags,CodeSnippets,FilteredSnippets,TestCode
2,"[windows, image, pdf]","[Convert taxes.pdf taxes.jpg \n, convert -size...",[convert -size 120x120 taxes.jpg.0 -geometry 1...,convert -size 120x120 taxes.jpg.0 -geometry 12...
5,"[sql, database]",[curs.execute('select * from people')\nfor row...,[curs.execute('select * from people')\nfor row...,curs.execute('select first_name from people')\...
7,"[arrays, iteration]",[ files = os.listdir(path) ...,[ files = os.listdir(path) ...,files = os.listdir(path) ...
8,"[arrays, iteration]","[#!/bin/python<br>bar in dict(Foo)<br>, has_ke...","[#!/bin/python<br>baz = dict([(key, value) for...","#!/bin/python<br>baz = dict([(key, value) for ..."
9,"[arrays, iteration]",[result = [obj for obj in listOfObjs if hasatt...,[result = [obj for obj in listOfObjs if hasatt...,result = [obj for obj in listOfObjs if hasattr...


In [49]:
oneHot = MultiLabelBinarizer()
oneHot.fit(df["Tags"])
num_labels = len(oneHot.classes_)

In [5]:
df["label"] = df["Tags"].map(lambda x: oneHot.transform([x])[0])
#Ez olyan sufnis szutyok megoldás hogy nem igaz de 3 óra alatt nem találtam jobbat úgyhogy szarok bele

In [6]:
data = Dataset.from_pandas(df[["TestCode", "label"]])
#data.remove_columns(['__index_level_0__'])

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
data = data.map(lambda examples: tokenizer(examples["TestCode"]), batched=True)

  0%|          | 0/487 [00:00<?, ?ba/s]

In [11]:
seed=2022
data = data.shuffle(seed=seed)

# 80% train, 20% test + validation
train_testvalid = data.train_test_split(test_size=0.2,seed=seed)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=seed)

# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

data

DatasetDict({
    train: Dataset({
        features: ['TestCode', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 388824
    })
    test: Dataset({
        features: ['TestCode', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 48603
    })
    valid: Dataset({
        features: ['TestCode', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 48603
    })
})

In [12]:
data.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [69]:
class CustomModel(torch.nn.Module):
    def __init__(self,checkpoint,num_labels): 
        super(CustomModel,self).__init__() 
        self.num_labels = num_labels 

        #Load Model with given checkpoint and extract its body
        self.model = model = transformers.AutoModel.from_pretrained(
            checkpoint,config=transformers.AutoConfig.from_pretrained(
                checkpoint, output_attentions=True,output_hidden_states=True))
        self.dropout = torch.nn.Dropout(0.1)
        #self.flatten = torch.nn.Flatten()
        self.classifier = torch.nn.Linear(768,num_labels) # load and initialize weights

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        #Extract outputs from the body
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        #Add custom layers
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

        logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses
    
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model=CustomModel(checkpoint='huggingface/CodeBERTa-small-v1',num_labels=num_labels).to(device)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    data["train"], shuffle=True, batch_size=2, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    data["valid"], batch_size=2, collate_fn=data_collator
)

In [72]:
from transformers import AdamW,get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

194412


In [73]:
from datasets import load_metric
metric = load_metric("f1")

In [74]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)
            
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
        progress_bar_eval.update(1)
    
    print(metric.compute())

  0%|          | 0/194412 [00:00<?, ?it/s]

  0%|          | 0/24302 [00:00<?, ?it/s]

ValueError: Expected input batch_size (2) to match target batch_size (412).

In [61]:
data["train"]["label"][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])